In [410]:
import pandas as pd
import numpy as np
import time
import xlrd

# `maritime_freight_unloaded.csv`

In [411]:
data = pd.read_csv('data/raw/maritime_freight_unloaded.csv')

print(f"Shape is: {data.shape}")
data.head()

Shape is: (6019, 11)


,country,country_code,series,seriescode,measure,period,D1,D2,D3,data,footnoteref
0,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2017SEP,NaN,NaN,NaN,662.0,NaN
1,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016DEC,NaN,NaN,NaN,628.0,NaN
2,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016SEP,NaN,NaN,NaN,584.0,NaN
3,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016OCT,NaN,NaN,NaN,748.0,NaN
4,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016YEAR,NaN,NaN,NaN,575.0,NaN


In [412]:
data.drop(['D1', 'D2', 'D3', 'footnoteref', 'measure', 'seriescode'], axis=1, inplace=True)
data.head()

,country,country_code,series,period,data
0,Latvia,428,Transport - International maritime freight unl...,2017SEP,662.0
1,Latvia,428,Transport - International maritime freight unl...,2016DEC,628.0
2,Latvia,428,Transport - International maritime freight unl...,2016SEP,584.0
3,Latvia,428,Transport - International maritime freight unl...,2016OCT,748.0
4,Latvia,428,Transport - International maritime freight unl...,2016YEAR,575.0


In [413]:
data = data[~data['data'].isna()]
data.isna().sum()

country         0
country_code    0
series          0
period          0
data            0
dtype: int64

In [414]:
data = data[(data['period'].str.contains('YEAR') == False) & (data['period'].str.contains('Q1') == False) & (data['period'].str.contains('Q2') == False) & (data['period'].str.contains('Q3') == False) & (data['period'].str.contains('Q4') == False)]

print(f"Shape is: {data.shape}")
data.head()

Shape is: (5136, 5)


,country,country_code,series,period,data
0,Latvia,428,Transport - International maritime freight unl...,2017SEP,662.0
1,Latvia,428,Transport - International maritime freight unl...,2016DEC,628.0
2,Latvia,428,Transport - International maritime freight unl...,2016SEP,584.0
3,Latvia,428,Transport - International maritime freight unl...,2016OCT,748.0
5,Latvia,428,Transport - International maritime freight unl...,2016JAN,423.0


In [415]:
def convert_date(cell):
    cell = cell.strip()
    year = cell[:4]
    month = cell[4:]
    return '-'.join([year, month])

data['period'] = data['period'].apply(convert_date)
data["period"] = pd.to_datetime(data["period"], format="%Y-%b")

data.dtypes

country                 object
country_code            object
series                  object
period          datetime64[ns]
data                   float64
dtype: object

In [416]:
data.head()

,country,country_code,series,period,data
0,Latvia,428,Transport - International maritime freight unl...,2017-09-01,662.0
1,Latvia,428,Transport - International maritime freight unl...,2016-12-01,628.0
2,Latvia,428,Transport - International maritime freight unl...,2016-09-01,584.0
3,Latvia,428,Transport - International maritime freight unl...,2016-10-01,748.0
5,Latvia,428,Transport - International maritime freight unl...,2016-01-01,423.0


In [417]:
data['series'].value_counts(dropna=False)

Transport - International maritime freight unloaded    2567
Transport - International maritime freight loaded      2523
Transport - International maritime freight total         46
Name: series, dtype: int64

In [418]:
data.rename(columns={
    'series' : 'freight_type',
    'data' : 'value_thousand_metric_tons'
    }, inplace=True)

data['freight_type'] = data['freight_type'].map({
    'Transport - International maritime freight unloaded' : 'unloaded',
    'Transport - International maritime freight loaded' : 'loaded',
    'Transport - International maritime freight total' : 'total' 
})

data.head()

,country,country_code,freight_type,period,value_thousand_metric_tons
0,Latvia,428,unloaded,2017-09-01,662.0
1,Latvia,428,unloaded,2016-12-01,628.0
2,Latvia,428,unloaded,2016-09-01,584.0
3,Latvia,428,unloaded,2016-10-01,748.0
5,Latvia,428,unloaded,2016-01-01,423.0


In [419]:
countries_data = pd.read_csv('data/cleaned/country_with_codes.csv')

data.drop('country_code', axis=1, inplace=True)

data = pd.merge(data, countries_data, left_on="country", right_on="country", how="left")

data.drop('country', axis=1, inplace=True)

data.head()

,freight_type,period,value_thousand_metric_tons,country_code
0,unloaded,2017-09-01,662.0,LVA
1,unloaded,2016-12-01,628.0,LVA
2,unloaded,2016-09-01,584.0,LVA
3,unloaded,2016-10-01,748.0,LVA
4,unloaded,2016-01-01,423.0,LVA


In [420]:
data.to_csv('data/cleaned/maritime_freight.csv', index=False)

# `SYB65_123_202209_Total Imports Exports and Balance of Trade.csv`

In [421]:
data = pd.read_csv('data/raw/SYB65_123_202209_Total Imports Exports and Balance of Trade.csv')
data.columns = ['Region/Country/Area', 'Total imports, exports and balance of trade', 'Year', 'Series', 'System of trade', 'System of trade footnote', 'Value', 'Footnotes', 'Source']

data = data.drop(data.index[0])

print(f"Shape is: {data.shape}")
data.head()

Shape is: (5059, 9)


,Region/Country/Area,"Total imports, exports and balance of trade",Year,Series,System of trade,System of trade footnote,Value,Footnotes,Source
1,1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),NaN,NaN,"5,099,057",NaN,"United Nations Statistics Division, New York, ..."
2,1,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),NaN,NaN,"10,577,013",NaN,"United Nations Statistics Division, New York, ..."
3,1,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),NaN,NaN,"15,275,115",NaN,"United Nations Statistics Division, New York, ..."
4,1,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),NaN,NaN,"16,482,346",NaN,"United Nations Statistics Division, New York, ..."
5,1,"Total, all countries or areas",2019,Imports CIF (millions of US dollars),NaN,NaN,"18,955,484",NaN,"United Nations Statistics Division, New York, ..."


In [422]:
data.drop(['Region/Country/Area', 'System of trade', 'System of trade footnote', 'Footnotes', 'Source'], axis=1, inplace=True)

print(f"Shape is: {data.shape}")
data.head()

Shape is: (5059, 4)


,"Total imports, exports and balance of trade",Year,Series,Value
1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),"5,099,057"
2,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),"10,577,013"
3,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),"15,275,115"
4,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),"16,482,346"
5,"Total, all countries or areas",2019,Imports CIF (millions of US dollars),"18,955,484"


In [423]:
data['Series'].value_counts()

Balance imports/exports (millions of US dollars)    1687
Imports CIF (millions of US dollars)                1686
Exports FOB (millions of US dollars)                1686
Name: Series, dtype: int64

In [424]:
data.rename(columns={
    'Total imports, exports and balance of trade' : 'total_imports_exports_and_balance_trade',
    'Year' : 'year',
    'Series' : 'trade_type',
    'Value' : 'value_million_usd'
    }, inplace=True)

data['trade_type'] = data['trade_type'].map({
    'Balance imports/exports (millions of US dollars)' : 'balance_import_export',
    'Imports CIF (millions of US dollars)' : 'imports',
    'Exports FOB (millions of US dollars)' : 'exports' 
})

data.head()

,total_imports_exports_and_balance_trade,year,trade_type,value_million_usd
1,"Total, all countries or areas",1995,imports,"5,099,057"
2,"Total, all countries or areas",2005,imports,"10,577,013"
3,"Total, all countries or areas",2010,imports,"15,275,115"
4,"Total, all countries or areas",2015,imports,"16,482,346"
5,"Total, all countries or areas",2019,imports,"18,955,484"


In [425]:
data["year"] = pd.to_datetime(data["year"], format="%Y")
data["value_million_usd"] = data["value_million_usd"].str.replace(',', '').astype(int)

data.head()

,total_imports_exports_and_balance_trade,year,trade_type,value_million_usd
1,"Total, all countries or areas",1995-01-01,imports,5099057
2,"Total, all countries or areas",2005-01-01,imports,10577013
3,"Total, all countries or areas",2010-01-01,imports,15275115
4,"Total, all countries or areas",2015-01-01,imports,16482346
5,"Total, all countries or areas",2019-01-01,imports,18955484


In [426]:
data = pd.merge(data, countries_data, left_on="total_imports_exports_and_balance_trade", right_on="country", how="left")

data.drop('country', axis=1, inplace=True)

data.head()

,total_imports_exports_and_balance_trade,year,trade_type,value_million_usd,country_code
0,"Total, all countries or areas",1995-01-01,imports,5099057,NaN
1,"Total, all countries or areas",2005-01-01,imports,10577013,NaN
2,"Total, all countries or areas",2010-01-01,imports,15275115,NaN
3,"Total, all countries or areas",2015-01-01,imports,16482346,NaN
4,"Total, all countries or areas",2019-01-01,imports,18955484,NaN


In [427]:
data.to_csv('data/cleaned/import_export_balance_trade.csv', index=False)

# `UN_paper_pulp_import_export.csv`

In [428]:
data = pd.read_csv('data/raw/UN_paper_pulp_import_export.csv')

print(f"Shape is: {data.shape}")
data.head()

Shape is: (38183, 6)


,Country or Area,Element,Year,Unit,Value,Value Footnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,233.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,646.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,140.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,45.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,291.0,Im


In [429]:
data.columns = [col.replace('or', '').replace('  ', '_').replace(' ', '').lower() for col in data.columns]
data.head()

,country_area,element,year,unit,value,valuefootnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,233.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,646.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,140.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,45.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,291.0,Im


In [430]:
data.drop('valuefootnotes', axis=1, inplace=True)

data.dropna(axis=0, inplace=True)

data["year"] = pd.to_datetime(data["year"], format="%Y")
data['value'] = data['value'].astype(int)

In [431]:
data = pd.merge(data, countries_data, left_on="country_area", right_on="country", how="left")

data.drop('country', axis=1, inplace=True)

data.head()

,country_area,element,year,unit,value,country_code
0,Afghanistan,Import Quantity,2020-01-01,tonnes,233,AFG
1,Afghanistan,Import Quantity,2019-01-01,tonnes,646,AFG
2,Afghanistan,Import Quantity,2018-01-01,tonnes,140,AFG
3,Afghanistan,Import Quantity,2017-01-01,tonnes,45,AFG
4,Afghanistan,Import Quantity,2016-01-01,tonnes,291,AFG


In [432]:
data.to_csv('data/cleaned/UN_paper_pulp_import_export.csv', index=False)

# `UN_wood_pulp_import_export.csv`

In [433]:
data = pd.read_csv('data/raw/UN_wood_pulp_import_export.csv')

print(f"Shape is: {data.shape}")
data.head()

Shape is: (36816, 6)


,Country or Area,Element,Year,Unit,Value,Value Footnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,245.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,622.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,127.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,32.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,167.0,Im


In [434]:
data.columns = [col.replace('or', '').replace('  ', '_').lower() for col in data.columns]
data.head()

,country_area,element,year,unit,value,value footnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,245.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,622.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,127.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,32.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,167.0,Im


In [435]:
data.drop('value footnotes', axis=1, inplace=True)

data.dropna(axis=0, inplace=True)

data["year"] = pd.to_datetime(data["year"], format="%Y")
data['value'] = data['value'].astype(int)

In [436]:
data.dtypes

country_area            object
element                 object
year            datetime64[ns]
unit                    object
value                    int64
dtype: object

In [437]:
data = pd.merge(data, countries_data, left_on="country_area", right_on="country", how="left")

data.drop('country', axis=1, inplace=True)

data.head()

,country_area,element,year,unit,value,country_code
0,Afghanistan,Import Quantity,2020-01-01,tonnes,245,AFG
1,Afghanistan,Import Quantity,2019-01-01,tonnes,622,AFG
2,Afghanistan,Import Quantity,2018-01-01,tonnes,127,AFG
3,Afghanistan,Import Quantity,2017-01-01,tonnes,32,AFG
4,Afghanistan,Import Quantity,2016-01-01,tonnes,167,AFG


In [438]:
data.to_csv('data/cleaned/UN_wood_pulp_import_export.csv', index=False)

# `PULP AND PAPER CAPACITY BY COUNTRY 2020-2025.xls`

In [ ]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/PULP AND PAPER CAPACITY BY COUNTRY 2020-2025.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

wood_pulp_capacities = []
wood_pulp_market_pulps = []

for index in range(2, 9, 2):
        wood_pulp_capacities.append([sheet.cell_value(rowx=12, colx=index) for sheet in sheets])
        wood_pulp_market_pulps.append([sheet.cell_value(rowx=12, colx=index+1) for sheet in sheets])

paper_paper_boards = []
for index in range(4, 8):
        paper_paper_boards.append([sheet.cell_value(rowx=56, colx=index) for sheet in sheets])

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'wood_pulp_capacity_2020': wood_pulp_capacities[0],
    'wood_pulp_market_pulps_2020': wood_pulp_market_pulps[0],
    'paper_paperboard_capacity_2020' :paper_paper_boards[0],
    'wood_pulp_capacity_2021': wood_pulp_capacities[1],
    'wood_pulp_market_pulps_2021': wood_pulp_market_pulps[1],
    'paper_paperboard_capacity_2021' :paper_paper_boards[1],
    'wood_pulp_capacity_2022': wood_pulp_capacities[2],
    'wood_pulp_market_pulps_2022': wood_pulp_market_pulps[2],
    'paper_paperboard_capacity_2022' :paper_paper_boards[2],
    'wood_pulp_capacity_2023': wood_pulp_capacities[3],
    'wood_pulp_market_pulps_2023': wood_pulp_market_pulps[3],
    'paper_paperboard_capacity_2023' :paper_paper_boards[3],
})

print(f"Shape is: {data.shape}")
data.head()

In [ ]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

In [ ]:
data.to_csv('data/cleaned/PULP AND PAPER CAPACITY BY COUNTRY 2020-2025.csv', index=False)

# `PULP AND PAPER PRODUCTION BY COUNTRY 2020.xls`

In [ ]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/PULP AND PAPER PRODUCTION BY COUNTRY 2020.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

wood_pulp_productions = [sheet.cell_value(rowx=12, colx=2) for sheet in sheets]
wood_pulp_productions_market_pulps = [sheet.cell_value(rowx=12, colx=3) for sheet in sheets]

paper_paper_boards = [sheet.cell_value(rowx=56, colx=2) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'wood_pulp_production_2020': wood_pulp_productions,
    'wood_pulp_market_pulps_2020': wood_pulp_productions_market_pulps,
    'paper_paperboard_production_2020' : paper_paper_boards
})

print(f"Shape is: {data.shape}")
data.head()

In [ ]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

In [ ]:
data.to_csv('data/cleaned/PULP AND PAPER PRODUCTION BY COUNTRY 2020.csv', index=False)

# `PULP_AND_PAPER_PRODUCTION_BY_COUNTRY_2021.xls`

In [ ]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/PULP_AND_PAPER_PRODUCTION_BY_COUNTRY_2021.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

wood_pulp_productions = [sheet.cell_value(rowx=12, colx=2) for sheet in sheets]
wood_pulp_productions_market_pulps = [sheet.cell_value(rowx=12, colx=3) for sheet in sheets]

paper_paper_boards = [sheet.cell_value(rowx=56, colx=2) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'wood_pulp_production_2021': wood_pulp_productions,
    'wood_pulp_market_pulps_2021': wood_pulp_productions_market_pulps,
    'paper_paperboard_production_2021' : paper_paper_boards
})

print(f"Shape is: {data.shape}")
data.head()

In [ ]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

In [ ]:
data.to_csv('data/cleaned/PULP_AND_PAPER_PRODUCTION_BY_COUNTRY_2021.csv', index=False)

# `UTILIZATION OF RECOVERED PAPER BY COUNTRY 2020.xls`

In [ ]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/UTILIZATION OF RECOVERED PAPER BY COUNTRY 2020.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

recovered_paper_totals = [sheet.cell_value(rowx=16, colx=1) for sheet in sheets]
mixed_grades = [sheet.cell_value(rowx=16, colx=2) for sheet in sheets]
corrugated_solid_containers = [sheet.cell_value(rowx=16, colx=3) for sheet in sheets]
newspapers = [sheet.cell_value(rowx=16, colx=4) for sheet in sheets]
wood_frees = [sheet.cell_value(rowx=16, colx=5) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'recovered_paper_totals_2020': recovered_paper_totals,
    'mixed_grades_2020': mixed_grades,
    'corrugated_solid_containers_2020' : corrugated_solid_containers,
    'newspapers_2020' : newspapers,
    'wood_free_2020' : wood_frees
})

print(f"Shape is: {data.shape}")
data.head()

In [ ]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

In [ ]:
data.to_csv('data/cleaned/UTILIZATION OF RECOVERED PAPER BY COUNTRY 2020.csv', index=False)

# `UTILIZATION_OF_RECOVERED_PAPER_BY_COUNTRY_2021.xls`

In [ ]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/UTILIZATION_OF_RECOVERED_PAPER_BY_COUNTRY_2021.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

recovered_paper_totals = [sheet.cell_value(rowx=16, colx=1) for sheet in sheets]
mixed_grades = [sheet.cell_value(rowx=16, colx=2) for sheet in sheets]
corrugated_solid_containers = [sheet.cell_value(rowx=16, colx=3) for sheet in sheets]
newspapers = [sheet.cell_value(rowx=16, colx=4) for sheet in sheets]
wood_frees = [sheet.cell_value(rowx=16, colx=5) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'recovered_paper_totals_2021': recovered_paper_totals,
    'mixed_grades_2021': mixed_grades,
    'corrugated_solid_containers_2021' : corrugated_solid_containers,
    'newspapers_2021' : newspapers,
    'wood_free_2021' : wood_frees
})

print(f"Shape is: {data.shape}")
data.head()

In [ ]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

In [ ]:
data.to_csv('data/cleaned/UTILIZATION_OF_RECOVERED_PAPER_BY_COUNTRY_2021.csv', index=False)